## Your guide to <img src="images/logo.png" alt="autoSKZCAM" style="display: inline; width: 300px; vertical-align: middle; margin-top: -10px;"/>

In this notebook, we will use the example of CO on MgO(001) to illustrate how autoSKZCAM can be used to get the adsorption enthalpy accurately and efficiently. It requires only the execution of a few simple functions, all of which are found under ``autoSKZCAM.recipes``. We will go over the set of functions to execute in a chronological order, starting with using the SKZCAM protocol to obtain the interaction energy, before going over the DFT ensemble.

We show an example python script that performs and analyses all of the calculations. This would typically be done in the following stages:
``skzcam_initialize`` &rarr; ``skzcam_eint_flow`` &rarr; ``skzcam_analyse`` &rarr; ``dft_ensemble_flow`` &rarr; ``dft_analyse`` &rarr; ``get_final_autoSKZCAM_Hads``. This first cell highlights the simplicity of performing autoSKZCAM. We give detailed explanation of each function in the rest of the notebook.

In [9]:
from __future__ import annotations

import pandas as pd  # Used to show results in a nice format
from data.misc import (
    OniomInfo,
    adsorbate,
    adsorbate_slab_gen_func,
    slab_gen_func,
    unit_cell,
    xc_ensemble,
)

from autoSKZCAM.recipes import (
    dft_ensemble_analyse,
    dft_ensemble_flow,
    get_final_autoSKZCAM_Hads,
    skzcam_analyse,
    skzcam_eint_flow,
    skzcam_initialize,
)

# Create the Embedded Cluster using our ChemShell wrapper
EmbeddedCluster = skzcam_initialize(
    adsorbate_indices=[0, 1],
    slab_center_indices=[32],
    atom_oxi_states={"Mg": 2, "O": -2},
    adsorbate_slab_file="data/POSCAR_CO_on_MgO",
    pun_filepath="data/ChemShell_EmbeddedCluster.pun",
)

# Generate the SKZCAM cluster and then run the necessary wave-function calculations on them
skzcam_eint_flow(
    EmbeddedCluster=EmbeddedCluster, OniomInfo=OniomInfo, calc_dir="./calc_dir"
)

# Analyse the calculations
skzcam_analysis = skzcam_analyse(
    calc_dir="./calc_dir", embedded_cluster_npy_path="data/embedded_cluster.npy"
)

# Run the DFT ensemble QuAcc workflow
dft_ensemble_calcs = dft_ensemble_flow(
    xc_ensemble=xc_ensemble,
    calc_dir="./dft_calc_dir",
    vib_xc_ensemble=["PBE-D2-Ne", "revPBE-D4", "vdW-DF", "rev-vdW-DF2"],
    geom_error_xc="revPBE-D4",
    slab_gen_func=slab_gen_func,
    adsorbate_slab_gen_func=adsorbate_slab_gen_func,
)

# Analyse the DFT calculations
dft_ensemble_analysis = dft_ensemble_analyse(
    calc_dir="dft_calc_dir",
    xc_ensemble=xc_ensemble,
    geom_error_xc="revPBE-D4",
    vib_xc_ensemble=["PBE-D2-Ne", "revPBE-D4", "vdW-DF", "rev-vdW-DF2"],
    freeze_surface_vib=True,
    temperature=61,
)

# Gather the DFT and SKZCAM analysis together
final_Hads = get_final_autoSKZCAM_Hads(
    skzcam_eint_analysis=skzcam_analysis, dft_ensemble_analysis=dft_ensemble_analysis
)


pd.DataFrame(
    {
        method: {
            "Contribution [meV]": int(round(value[0])),
            "Error [meV]": int(round(value[1])),
        }
        for method, value in final_Hads.items()
    }
).T

,Contribution [meV],Error [meV]
Bulk MP2,-165,0
Delta_Basis and Delta_Core,-27,6
FSE Error,0,3
DeltaCC,-10,1
Overall Eint,-203,7
DFT Erlx,8,20
DFT DeltaH,24,3
Final Hads,-170,21


# Part A: Interaction energies with the SKZCAM protocol

## 1. Initialize the embedded cluster with: `skzcam_initialize`

In [2]:
EmbeddedCluster = skzcam_initialize(
    adsorbate_indices=[0, 1],
    slab_center_indices=[32],
    atom_oxi_states={"Mg": 2, "O": -2},
    adsorbate_slab_file="data/POSCAR_CO_on_MgO",
    pun_filepath="data/ChemShell_EmbeddedCluster.pun",
)

Initializes the SKZCAM protocol to generate the embedded clusters. ChemShell will be executed if it hasn't already been run to produce a .pun file containing the embedded cluster positions. It is required that you have installed py-ChemShell v20.0.0 and up at: https://chemshell.org/licence/. The main output of `skzcam_initialize`` is the EmbeddedCluster object, which contains an ASE Atoms object of the electrostatic embedded cluster generated by ChemShell. See https://www.chemshell.org/static_files/py-chemshell/manual/build/html/cluster.html for more information and an explanation of ChemShell specific inputs.

The code executed above assumes ChemShell has been run prior to produce a .pun file called `ChemShell_EmbeddedCluster.pun`. Use the code below if ChemShell has not been executed already for this system:

```python
EmbeddedCluster = skzcam_initialize(adsorbate_indices =[0,1],slab_center_indices=[32],atom_oxi_states = {'Mg':2, 'O':-2},adsorbate_slab_file='POSCAR_CO_on_MgO', run_chemshell=True)
```

<div style="float: right; margin: 0 0 10px 10px;">
  <img src="images/co_on_mgo.png" alt="Alt text" width="200"/>
</div>

**Parameters**  
- **<code>adsorbate_indices</code>** (`list[int]`, **required**):  
  Indices of the atoms that make up the adsorbate molecule.

- **<code>slab_center_indices</code>** (`list[int]`, **required**):  
  Indices of the atoms that make up the “center” of the slab right beneath the adsorbate.

- **<code>atom_oxi_states</code>** (`dict[ElementStr, int]`, **required**):  
  Dictionary with element symbols as keys and their oxidation states as values.

- **<code>adsorbate_slab_file</code>** (`str | Path`, **required**):  
  Path to the file containing the adsorbate molecule on the surface slab (any format readable by ASE).

- **<code>pun_filepath</code>** (`str | Path`, default: `./ChemShell_EmbeddedCluster.pun`):  
  Path to the `.pun` file containing coordinates/charges if already generated by ChemShell. If `None`, ChemShell will be called to create this file.

- **<code>run_chemshell</code>** (`bool`, default: `False`):  
  If `True`, runs ChemShell to create or update the `.pun` file if it doesn’t already exist.

- **<code>chemsh_radius_active</code>** (`float`, default: `40.0`):  
  Radius of the active region (in Å) for charge fitting to ensure correct Madelung potential. Can be relatively large.

- **<code>chemsh_radius_cluster</code>** (`float`, default: `60.0`):  
  Total radius (in Å) of the embedded cluster.

- **<code>chemsh_bq_layer</code>** (`float`, default: `6.0`):  
  Height (in Å) above the surface at which additional fitting point charges are placed to improve the electrostatic potential near the adsorbate.

- **<code>write_xyz_file</code>** (`bool`, default: `False`):  
  If `True`, writes out the XYZ file for the final cluster.

**Returns**
<code>CreateEmbeddedCluster</code>

An object representing the constructed embedded cluster. See [here](https://github.com/benshi97/autoSKZCAM/blob/main/src/autoSKZCAM/embed.py) for more details. 
<details>
  <summary>Click for object attributes</summary>

- <code>adsorbate_indices</code>: Indices of atoms in the adsorbate.
- <code>slab_center_indices</code>: Indices of atoms at the center of the slab.
- <code>slab_indices</code>: Indices of atoms in the slab (initialized as <code>None</code>).
- <code>atom_oxi_states</code>: Oxidation states of atoms.
- <code>adsorbate_slab_file</code>: File containing the geometry of the adsorbate-slab complex.
- <code>pun_filepath</code>: Filepath to the <code>.pun</code> file for ChemShell.
- <code>skzcam_calcs</code>: Dictionary for calculator information for SKZCAM clusters (<code>None</code> by default).
- <code>OniomInfo</code>: Dictionary for information on ONIOM layers (<code>None</code> by default).
- <code>adsorbate</code>: <code>Atoms</code> object representing the adsorbate (<code>None</code> by default).
- <code>slab</code>: <code>Atoms</code> object representing the slab (<code>None</code> by default).
- <code>adsorbate_slab</code>: <code>Atoms</code> object for the adsorbate-slab complex (<code>None</code> by default).
- <code>adsorbate_slab_embedded_cluster</code>: <code>Atoms</code> object for the embedded adsorbate-slab cluster (<code>None</code> by default).
- <code>slab_embedded_cluster</code>: <code>Atoms</code> object for the embedded slab cluster (<code>None</code> by default).
- <code>quantum_cluster_indices_set</code>: List of quantum cluster indices (<code>None</code> by default).
- <code>ecp_region_indices_set</code>: List of indices for the ECP region (<code>None</code> by default).
- <code>adsorbate_vector_from_slab</code>: Vector representing the position of the adsorbate relative to the slab (calculated during initialization).
- <code>center_position</code>: Center position of the cluster (calculated during initialization).

</details>

## 2. Perform and run calculations for the SKZCAM protocol with: `skzcam_eint_flow`

In [3]:
from data.OniomInfo import OniomInfo

skzcam_eint_flow(EmbeddedCluster=EmbeddedCluster, OniomInfo=OniomInfo)

<div style="display: flex; align-items: flex-start;">

  <!-- Left Column: Text and Image -->
  <div style="width: 70%; padding-right: 10px;">
    <p>

<div style="text-align: center; margin: 20px 0;">
  <img src="images/ONIOM.png" alt="Alt text" width="900" style="display: block; margin: auto;"/>
</div>

This function performs multiple tasks, first generating the series of clusters according to the SKZCAM protocol, then performing all the necessary high-level calculations on these clusters. It is also possible to simply generate the necessary input files in a chosen directory **calc_dir**, for which you can subsequently run on your chosen computing cluster. This functionality can be turned on using **dry_run**=`True`

**Parameters**  
- **<code>EmbeddedCluster</code>** (`CreateEmbeddedCluster`, **required**):  
  The `CreateEmbeddedCluster` object containing the embedded cluster. This is initialized using the `skzcam_initialize()` function previously.

- **<code>OniomInfo</code>** (`dict[str, OniomLayerInfo]`, **required**):  
  A dictionary containing the information about the ONIOM layers (see the right).

- **<code>kwargs</code>** (`dict[str,Any]`, optional):  
  Additional keyword arguments to pass to the `autoSKZCAM.recipes_skzcam.skzcam_generate_job()` and `autoSKZCAM.recipes_skzcam.skzcam_calculate_job()` functions. Particularly useful ones are `dry_run`, `use_quacc`, `calc_dir` and `write_clusters`. See below for a summary of the kwargs.

  </p>
<details>
  <summary>Click for summary of the kwargs:</summary>

From <code>autoSKZCAM.recipes_skzcam.skzcam_calculate_job()</code>:
- <code>dry_run</code> (<code>bool</code>, default: <code>False</code>): Whether to simply write out the input files.
- <code>use_quacc</code> (<code>bool</code>, default: <code>False</code>): Whether to use QuAcc to perform and manage the calculations.
- <code>calc_dir</code> (<code>str | Path</code>, default: <code>"./calc_dir"</code>): The directory to place all the calculation results

From <code>autoSKZCAM.recipes_skzcam.skzcam_generate_job()</code>:
- <code>max_cluster_num</code> (<code>int</code>, default: <code>10</code>): The maximum number of quantum clusters to be created.
- <code>shell_width</code> (<code>float</code>, default: <code>0.1</code>): Defines the distance between atoms within shells; this is the maximum distance between any two atoms within the shell.
- <code>bond_dist</code> (<code>float</code>, default: <code>2.5</code>): The distance within which an anion is considered to be coordinating a cation.
- <code>ecp_dist</code> (<code>float</code>, default: <code>6.0</code>): The distance from edges of the quantum cluster to define the ECP region.
- <code>write_clusters</code> (<code>bool</code>, default: <code>False</code>): If <code>True</code>, the quantum clusters generated by the SKZCAM protocol will be written to a file.
- <code>write_clusters_path</code> (<code>str | Path</code>, default: <code>"."</code>): The path to the file where the quantum clusters will be written.
- <code>write_include_ecp</code> (<code>bool</code>, default: <code>False</code>): If <code>True</code>, the ECP region will be included in the quantum clusters.

</details>

<p>
    
**Returns**  
<code>None</code>

The main additional input is `OniomInfo` (see right pane), which is a dictionary describing the high-level (hl) and low-level (ll) calculation of each ONIOM layer. See https://pubs.acs.org/doi/10.1021/cr5004419 for more details. OniomInfo is a nested dictionary in the order `ONIOM layer name` &rarr; `ll` or `hl` level of theory &rarr; `parameters`. The `ONIOM layer name` can be any type of string, but it must include either `bulk`, `fse` or `delta` in the name depending on the function of the layer. For example `Extrapolated Bulk method` would give an extrapolated bulk number for the chosen method. For this type of layer, only the `hl` calculation needs to be provided. A `ONIOM layer name` that contains `delta`, such as `DeltaCC`, would calculate a difference between the `ll` and `hl` levels; this can be used for calculating basis set, coupled cluster or core corrections. Finally, a layer with `FSE` would calculate the finite size error and the main difference should be in the **max_cluster_num** between the `ll` and `hl` parameters. 


The `parameters` that can be used are:
- **<code>method</code>** (`str`) - The type of calculation to perform, options for `code=mrcc` are `LMP2`,`LNO-CCSD(T)`,`MP2`,`CCSD(T)` while options for `code=orca` are `DLPNO-MP2`,`DLPNO-CCSD(T)`,`MP2`,`CCSD(T)`. It is possible to specify your own method (e.g., RPA or CISD), but that require giving the code-specific options in **code_inputs**.
- **<code>frozen_core</code>** (`str`) - The type of frozen core treatment to use for the correlated wavefunction calculations. Either `semicore` or `valence` are used. The `semicore` can be found here: https://www.faccts.de/docs/orca/6.0/manual/contents/detailed/frozencore.html
- **<code>basis</code>** (`str`) - This specifies the size of the basis set in terms of $\zeta$. Options are: `DZ`,`TZ`,`QZ`,`5Z`,`CBS(DZ//TZ)`,`CBS(TZ//QZ)`,`CBS(QZ//5Z)`. The CBS parameter would create two calculations for each of the enclosed basis sets. The default is to use the cc-pVXZ or cc-pwCVXZ (for **frozen_core**=`semicore`) basis sets on the cations and aug-cc-pVXZ basis sets on the rest.
- **<code>max_cluster_num</code>** (`int`) - The number of clusters from the SKZCAM protocol to be used for this calculation.
- **<code>code</code>** - Currently, `mrcc` and `orca` are supported.
- **<code>code_inputs</code>** (`dict[str,str]`, **optional**) - Code specific inputs. For **code**=`orca`, the two options are `orcasimpleinput` and `orcablocks`, for example it can be provided as ```{"orcasimpleinput": "DLPNO-CCSD TightPNO def2/J", "orcablocks": "%nprocs 2 end\n%maxcore 2000\n"}``` . For **code**=`mrcc`, the dictionary can be provided as a list of keys and values, such as ```{"calc": "dRPA75", "mem": "20000MB","symm":"off"}```
- **<code>element_info</code>** (`dict[str,str]`, **optional**) - A dictionary with elements as keys which gives the (1) number of core electrons as `core`, (2) basis set as `basis`, (3) effective core potential as `ecp`, (4) resolution-of-identity/density-fitting auxiliary basis set for DFT/HF calculations as `ri_scf_basis` and (5) resolution-of-identity/density-fitting for correlated wave-function methods as `ri_cwft_basis`. An example element_info is as follows: ```{'C': {"basis": "def2-SVP", "ri_scf_basis": "def2/J", "ri_cwft_basis": "def2-SVP/C"}``` which changes the basis sets for the C atom from the defaults. For CBS calculations, it can be specified as e.g., `"basis": "def2-SVP/def2-TZVPP"`


where the first gives the name of the layer (i.e., what type of correction it involves). A key with the name `Delta` is a delta correction and as such contains both a "ll" and "hl" key.

</p>
  </div>

  <!-- Right Column: Python Code -->
<div style="width: 30%; padding-left: 10px; background-color: #f0f0f0; padding: 15px; border-radius: 5px;">
<code>OniomInfo = {
    "Bulk MP2": {
        "ll": None,
        "hl": {
            "method": "MP2",
            "frozen_core": "valence",
            "basis": "CBS(DZ//TZ)",
            "max_cluster_num": 5,
            "code": "orca",
        },
    },
    "Delta_Basis and Delta_Core": {
        "ll": {
            "method": "MP2",
            "frozen_core": "valence",
            "basis": "CBS(DZ//TZ)",
            "max_cluster_num": 3,
            "code": "orca",
        },
        "hl": {
            "method": "MP2",
            "frozen_core": "semicore",
            "basis": "CBS(TZ//QZ)",
            "max_cluster_num": 3,
            "code": "orca",
        },
    },
    "FSE Error": {
        "ll": {
            "method": "MP2",
            "frozen_core": "valence",
            "basis": "DZ",
            "max_cluster_num": 5,
            "code": "orca",
        },
        "hl": {
            "method": "MP2",
            "frozen_core": "valence",
            "basis": "DZ",
            "max_cluster_num": 7,
            "code": "orca",
        },
    },
    "DeltaCC": {
        "ll": {
            "method": "LMP2",
            "frozen_core": "valence",
            "basis": "CBS(DZ//TZ)",
            "max_cluster_num": 3,
            "code": "mrcc",
        },
        "hl": {
            "method": "LNO-CCSD(T)",
            "frozen_core": "valence",
            "basis": "CBS(DZ//TZ)",
            "max_cluster_num": 3,
            "code": "mrcc",
        },
    },
}</code>
  </div>

</div>


## 3. Analyse the SKZCAM calculations with: `skzcam_analyse`

In [8]:
skzcam_analysis = skzcam_analyse(
    calc_dir="./calc_dir", embedded_cluster_npy_path="data/embedded_cluster.npy"
)
pd.DataFrame(
    {
        method: {
            "Contribution [meV]": int(round(value[0])),
            "Error [meV]": int(round(value[1])),
        }
        for method, value in skzcam_analysis.items()
    }
).T

,Contribution [meV],Error [meV]
Bulk MP2,-165,0
Delta_Basis and Delta_Core,-27,6
FSE Error,0,3
DeltaCC,-10,1
Overall Eint,-203,7


This function analyses the calculations performed in the previous step to give the final interaction energy from the SKZCAM protocol. If you have already executed `skzcam_eint_flow`, there should be a file called `embedded_cluster.npy` in the chosen `calc_dir` folder. You can alternatively directly specify the path to this file. Specifying `calc_dir` and `embedded_cluster.npy` should be sufficient to perform the analysis. We also give the option to instead supply `OniomInfo` and `EmbeddedCluster` in the event `embedded_cluster_npy_path` is not provided.

**Parameters**  
- **<code>calc_dir</code>** (<code>str | Path</code>, **required**):  
  The directory containing the calculations.

- **<code>embedded_cluster_npy_path</code>** (<code>Path | str | None</code>, default: <code>None</code>):  
  The path to the embedded cluster `.npy` object. If not provided, this will be inferred or generated.

- **<code>OniomInfo</code>** (<code>dict[str, OniomLayerInfo] | None</code>, default: <code>None</code>):  
  The dictionary containing the information about the ONIOM layers.

- **<code>EmbeddedCluster</code>** (<code>CreateEmbeddedCluster | None</code>, default: <code>None</code>):  
  The <code>CreateEmbeddedCluster</code> object containing the embedded cluster.

**Returns**  
- <code>dict[str, tuple[float, float]]</code>:  
  A dictionary containing the ONIOM layer as the key and a tuple as the value, where the tuple contains:
  - Contribution to the final interaction energy.
  - The associated error.


# Part B: Geometrical and thermal contributions with the DFT ensemble

## 1. Run entire DFT ensemble QuAcc workflow with: `dft_ensemble_flow`

In [4]:
from data.xc_ensemble import xc_ensemble
import numpy as np
from ase.build import sort, surface
from ase.constraints import FixAtoms
from ase.io import read

# To start, we need to load a CO molecule and a MgO conventional unit cell
adsorbate = read("data/POSCAR_CO")   # noqa F811
unit_cell = read("data/POSCAR_MgO")  # noqa F811


# Define the function which can convert the unit cell into a MgO(001) surface.
def slab_gen_func(unit_cell):
    surface_cell = sort(
        surface(unit_cell, (0, 0, 1), 2, vacuum=7.5, periodic=True) * (2, 2, 1)
    )

    fix_list = []
    for atom_idx in surface_cell:
        if atom_idx.position[2] < (np.max(surface_cell.get_positions()[:, 2]) - 3):
            fix_list += [atom_idx.index]

    c = FixAtoms(indices=fix_list)
    surface_cell.set_constraint(c)
    return surface_cell


# Define the function which can add the adsorbate and slab to create a CO molecule adsorbed on top of a Mg site.
def adsorbate_slab_gen_func(adsorbate, slab):
    maxzpos = np.max(slab.get_positions()[:, 2])
    top_Mg_index = next(
        atom.index
        for atom in slab
        if (abs(atom.position[2] - maxzpos) < 0.1 and atom.symbol == "Mg")
    )
    adsorbate.set_cell(slab.get_cell())
    adsorbate.set_pbc(slab.get_pbc())
    adsorbate.translate(
        slab[top_Mg_index].position - adsorbate.get_positions()[0] + np.array([0, 0, 2])
    )

    adsorbate_slab = adsorbate + slab
    slab_indices = slab.constraints[0].__dict__["index"]

    c = FixAtoms(indices=len(adsorbate) + slab_indices)
    adsorbate_slab.set_constraint(c)

    return adsorbate_slab


dft_ensemble_calcs = dft_ensemble_flow(
    xc_ensemble=xc_ensemble,
    calc_dir="./dft_calc_dir",
    vib_xc_ensemble=["PBE-D2-Ne", "revPBE-D4", "vdW-DF", "rev-vdW-DF2"],
    geom_error_xc="revPBE-D4",
    slab_gen_func=slab_gen_func,
    adsorbate_slab_gen_func=adsorbate_slab_gen_func,
)

<div style="display: flex; align-items: flex-start;">

  <!-- Left Column: Text and Image -->
  <div style="width: 60%; padding-right: 10px;">
    <p>

The calculations required for computing the geometrical relaxation (error) energy and the thermal contributions involves multiple steps, especially when involving an ensemble of XC functionals. We use the QuAcc computational materials science workflow library (see: https://github.com/Quantum-Accelerators/quacc) to handle all the individual components (see right panel).

**Parameters**  
- **<code>xc_ensemble</code>** (<code>dict[str, dict[str, Any]]</code>, **required**):  
  A dictionary containing the exchange-correlation (xc) functionals to be used as keys. The values are a dictionary with the VASP parameter as key and its value to define the xc functional. For example: {"PBE-D2-Ne": {"GGA": "PE", "IVDW": 1}, "vdW-DF": {"GGA": "RE", "AGGAC": 0.0, "LUSE_VDW": True, "LASPH": True} defines a DFT ensemble containing the PBE-D2[Ne] and vdW-DF functionals.

- **<code>vib_xc_ensemble</code>** (<code>list[str] | None</code>, default: <code>None</code>):  
  A list of xc functionals for which vibrational calculations will be performed. If <code>None</code>, vibrational calculations will be performed for all functionals.

- **<code>geom_error_xc</code>** (<code>str | None</code>, default: <code>None</code>):  
  The xc functional to use for calculating geometry errors. This is the functional used to generate the CO on MgO(001) slab for the SKZCAM interaction energy calculations.

- **<code>freeze_surface_vib</code>** (<code>bool</code>, default: <code>True</code>):  
  If <code>True</code>, all slab atoms will be frozen during vibrational calculations.

- **<code>job_params</code>** (<code>dict[str, dict[str, Any]] | None</code>, default: <code>None</code>):  
  A dictionary containing the job parameters for each of the jobs in the ensemble. If <code>None</code>, default job parameters will be used. The keys are: '01-molecule','02-unit_cell','03-slab','04-adsorbate_slab','05-adsorbate_slab_vib','06-molecule_vib','07-slab_vib','08-eint_adsorbate_slab','09-eint_adsorbate','10-eint_slab'. If you wanted to change the energy cutoff for the molecule relaxation in the 02-unit_cell relaxation, you would specify: {'02-unit_cell': {'encut': '600'}}.

- **<code>adsorbate</code>** (<code>Atoms | None</code>, default: <code>None</code>):  
  The adsorbate molecule to be used for the calculations.

- **<code>unit_cell</code>** (<code>Atoms | None</code>, default: <code>None</code>):  
  The unit cell of the solid to be used for the calculations.

- **<code>calc_dir</code>** (<code>str | Path</code>, default: <code>"./dft_calc_dir"</code>):  
  The directory where the calculations will be performed.

- **<code>slab_gen_func</code>** (<code>Callable[[Atoms], Atoms] | None</code>, default: <code>None</code>):  
  A function to generate the slab from the unit cell.

- **<code>adsorbate_slab_gen_func</code>** (<code>Callable[[Atoms], Atoms] | None</code>, default: <code>None</code>):  
  A function to generate the adsorbate-slab complex from the adsorbate and slab. It is important that the indices of the adsorbates are the first indices in the <code>Atoms</code> object, followed by the slab indices.

**Returns**  
- <code>dict[str, dict[str, VaspSchema | Schema]]</code>:  
  A dictionary containing the results of the DFT ensemble calculations for each functional in the ensemble. The key is the job calculation type, i.e., it can '01-molecule','02-unit_cell','03-slab','04-adsorbate_slab','05-adsorbate_slab_vib','06-molecule_vib','07-slab_vib','08-eint_adsorbate_slab','09-eint_adsorbate','10-eint_slab'. The calculations are stored as an ASE or VASP Schema.

</p>

<details>
  <summary>Click for example of a Schema:</summary>
<code>{'atoms': Atoms(symbols='CO', pbc=True, cell=[20.0, 20.0, 21.126328287]),
 'builder_meta': {'build_date': datetime.datetime(2025, 1, 27, 22, 17, 53, 416204, tzinfo=datetime.timezone.utc),
  'emmet_version': '0.84.6rc3',
  'pymatgen_version': '2025.1.9'},
 'chemsys': 'C-O',
 'composition': Composition('C1 O1'),
 'composition_reduced': Composition('C1 O1'),
 'density': 0.005504016679745712,
 'density_atomic': 4225.2656574,
 'dir_name': PosixPath('dft_calc_dir/01-molecule/PBE-D2-Ne'),
 'elements': [Element C, Element O],
 'formula_anonymous': 'AB',
 'formula_pretty': 'CO',
 'input_atoms': {'atoms': Atoms(symbols='CO', pbc=True, cell=[20.0, 20.0, 21.126328287]),
  'builder_meta': {'build_date': datetime.datetime(2025, 1, 27, 22, 17, 53, 413102, tzinfo=datetime.timezone.utc),
   'emmet_version': '0.84.6rc3',
   'pymatgen_version': '2025.1.9'},
  'chemsys': 'C-O',
  'composition': Composition('C1 O1'),
  'composition_reduced': Composition('C1 O1'),
  'density': 0.005504016679745712,
  'density_atomic': 4225.2656574,
  'elements': [Element C, Element O],
  'formula_anonymous': 'AB',
  'formula_pretty': 'CO',
  'nelements': 2,
  'nsites': 2,
  'symmetry': {'angle_tolerance': 5.0,
   'crystal_system': <CrystalSystem.tet: 'Tetragonal'>,
   'number': 99,
   'point_group': '4mm',
   'symbol': 'P4mm',
   'symprec': 0.1,
   'version': '2.5.0'},
  'volume': 8450.5313148},
 'nelements': 2,
 'nid': 'DESKTOP-Q5QPHVF.',
 'nsites': 2,
 'parameters': {},
 'quacc_version': '0.11.13',
 'results': {'energy': -14.79708994,
  'forces': array([[ 0.0e+00,  0.0e+00, -7.9e-05],
         [ 0.0e+00,  0.0e+00,  7.9e-05]]),
  'free_energy': -14.79708994,
  'stress': array([ 7.17773549e-06,  7.17773549e-06,  8.38858827e-06, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00])},
 'structure': Structure Summary
 Lattice
     abc : 20.0 20.0 21.126328287
  angles : 90.0 90.0 90.0
  volume : 8450.5313148
       A : 20.0 0.0 0.0
       B : 0.0 20.0 0.0
       C : 0.0 0.0 21.126328287
     pbc : True True True
 PeriodicSite: C (10.0, 10.0, 9.992) [0.5, 0.5, 0.4729]
 PeriodicSite: O (10.0, 10.0, 11.13) [0.5, 0.5, 0.5271],
 'symmetry': {'angle_tolerance': 5.0,
  'crystal_system': <CrystalSystem.tet: 'Tetragonal'>,
  'number': 99,
  'point_group': '4mm',
  'symbol': 'P4mm',
  'symprec': 0.1,
  'version': '2.5.0'},
 'volume': 8450.5313148}</code>

</details>

</div>
  <!-- Right Column: Image -->
  <div style="width: 30%; text-align: center;">
    <img src="images/dft_workflow.png" alt="Alt text" width="200" style="display: block; margin: auto;" />
  </div>

## 2. Analyse DFT calculations

In [6]:
dft_ensemble_analysis = dft_ensemble_analyse(
    calc_dir="dft_calc_dir",
    xc_ensemble=xc_ensemble,
    geom_error_xc="revPBE-D4",
    vib_xc_ensemble=["PBE-D2-Ne", "revPBE-D4", "vdW-DF", "rev-vdW-DF2"],
    freeze_surface_vib=True,
    temperature=61,
)
pd.DataFrame(
    {
        method: {
            "Contribution [meV]": int(round(value[0])),
            "Error [meV]": int(round(value[1])),
        }
        for method, value in dft_ensemble_analysis.items()
    }
).T

,Contribution [meV],Error [meV]
DFT Erlx,8,20
DFT DeltaH,24,3
